In [1]:
import time
import pandas as pd
from datetime import datetime
from appium import webdriver
from appium.options.android import UiAutomator2Options
from appium.webdriver.common.appiumby import AppiumBy
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# ===================== [설정 영역: 고용24] =====================
APP_PACKAGE = "kr.or.keis.mo"
APP_ACTIVITY = "kr.or.keis.mo.MainActivity"
DEVICE_NAME = "Galaxy S25"
APPIUM_SERVER_URL = "http://127.0.0.1:4723"

# 반복횟수
REPEAT_COUNT = 10

# 검색어 입력값
KEYWORD = "실업급여"

In [3]:
# 메인 테스트

def run_work24_search_test():
    options = UiAutomator2Options()
    options.device_name = DEVICE_NAME
    options.app_package = APP_PACKAGE
    options.app_activity = APP_ACTIVITY
    options.automation_name = "UiAutomator2"
    options.no_reset = True
    options.new_command_timeout = 300
    
    options.set_capability("connectHardwareKeyboard", True)

    print(f"--- 고용24(Work24) 검색 성능 측정 ({REPEAT_COUNT}회) 시작 ---")
    
    driver = None
    results = []

    try:
        driver = webdriver.Remote(APPIUM_SERVER_URL, options=options)
        wait = WebDriverWait(driver, 20)

        print("📱 앱 실행 및 메인 화면 대기 중...")
        # 메인 검색창이 뜰 때까지 대기 (바로 노출된다고 하셨으므로)
        wait.until(EC.visibility_of_element_located(
            (AppiumBy.XPATH, '//android.widget.EditText[@resource-id="top-topQueryMain"]')
        ))

        # ===================== 반복 측정 루프 =====================
        for i in range(1, REPEAT_COUNT + 1):
            try:
                print(f"\n[Running] {i}/{REPEAT_COUNT}회차 측정 진행 중...")

                # ---------------------------------------------------------
                # Step 1. 검색어 입력 (메인화면 검색창)
                # ---------------------------------------------------------
                search_input = wait.until(EC.visibility_of_element_located(
                    (AppiumBy.XPATH, '//android.widget.EditText[@resource-id="top-topQueryMain"]')
                ))
                
                search_input.click()
                search_input.clear()
                search_input.send_keys(KEYWORD)
                
                # ---------------------------------------------------------
                # Step 2. 검색 버튼 클릭 (측정 시작 T1)
                # ---------------------------------------------------------
                search_btn = driver.find_element(AppiumBy.XPATH, '//android.widget.Button[@resource-id="top-findSearchDataMain"]')
                
                # [Time Start]
                start_time = time.time() 
                search_btn.click()
                
                # ---------------------------------------------------------
                # Step 3. 결과 화면 검증 (측정 종료 T2)
                # ---------------------------------------------------------
                # "검색 결과" 텍스트가 포함된 뷰가 뜰 때까지 대기
                wait.until(EC.presence_of_element_located(
                    (AppiumBy.XPATH, '//android.view.View[contains(@text, "검색 결과")]')
                ))
                
                # [Time End]
                end_time = time.time()
                
                duration = end_time - start_time
                print(f"⏱️ {i}회차 소요 시간: {duration:.4f}초")

                results.append({
                    "회차": i,
                    "측정시각": datetime.now().strftime("%H:%M:%S"),
                    "검색어": KEYWORD,
                    "검색반응속도(초)": round(duration, 4)
                })

                # ---------------------------------------------------------
                # Step 4. 메인 화면 복귀 (뒤로 가기)
                # ---------------------------------------------------------
                print("🔙 하드웨어 뒤로가기 키 입력")
                driver.press_keycode(4) # Back Button

                # 메인 화면 복귀 확인 (다음 루프를 위해 메인 검색창 대기)
                wait.until(EC.visibility_of_element_located(
                    (AppiumBy.XPATH, '//android.widget.EditText[@resource-id="top-topQueryMain"]')
                ))
                
                time.sleep(1) # 안정화 대기

            except Exception as e:
                print(f"❌ {i}회차 실행 중 에러 발생: {e}")
                try:
                    driver.press_keycode(4) # 에러 시 뒤로가기 시도
                except:
                    pass

        # ===================== CSV 저장 로직 =====================
        print("\n" + "=" * 50)
        print("💾 결과 저장 중...")

        if results:
            df = pd.DataFrame(results)
            speeds = df["검색반응속도(초)"]
            
            summary = {
                "회차": "요약",
                "검색반응속도(초)": "",
                "평균(초)": round(speeds.mean(), 4),
                "최소(초)": round(speeds.min(), 4),
                "최대(초)": round(speeds.max(), 4),
                "표준편차(초)": round(speeds.std(ddof=1), 4) if len(speeds) >= 2 else 0.0
            }
            
            df = pd.concat([df, pd.DataFrame([summary])], ignore_index=True)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            file_name = f"work24_search_perf_{REPEAT_COUNT}runs_{timestamp}.csv"
            df.to_csv(file_name, index=False, encoding="utf-8-sig")

            print(f"✅ 저장 완료: {file_name}")
            print(df)
        else:
            print("ℹ️ 저장할 데이터가 없습니다.")

    except Exception as e:
        print(f"⛔ 치명적 오류: {e}")

    finally:
        if driver:
            driver.quit()

In [4]:
# 테스트 실행
if __name__ == "__main__":
    run_work24_search_test()

--- 고용24(Work24) 검색 성능 측정 (10회) 시작 ---
📱 앱 실행 및 메인 화면 대기 중...

[Running] 1/10회차 측정 진행 중...
⏱️ 1회차 소요 시간: 1.5137초
🔙 하드웨어 뒤로가기 키 입력

[Running] 2/10회차 측정 진행 중...
⏱️ 2회차 소요 시간: 2.1937초
🔙 하드웨어 뒤로가기 키 입력

[Running] 3/10회차 측정 진행 중...
⏱️ 3회차 소요 시간: 1.9320초
🔙 하드웨어 뒤로가기 키 입력

[Running] 4/10회차 측정 진행 중...
⏱️ 4회차 소요 시간: 2.1835초
🔙 하드웨어 뒤로가기 키 입력

[Running] 5/10회차 측정 진행 중...
⏱️ 5회차 소요 시간: 2.1788초
🔙 하드웨어 뒤로가기 키 입력

[Running] 6/10회차 측정 진행 중...
⏱️ 6회차 소요 시간: 2.1516초
🔙 하드웨어 뒤로가기 키 입력

[Running] 7/10회차 측정 진행 중...
⏱️ 7회차 소요 시간: 2.1035초
🔙 하드웨어 뒤로가기 키 입력

[Running] 8/10회차 측정 진행 중...
⏱️ 8회차 소요 시간: 2.1211초
🔙 하드웨어 뒤로가기 키 입력

[Running] 9/10회차 측정 진행 중...
⏱️ 9회차 소요 시간: 2.1939초
🔙 하드웨어 뒤로가기 키 입력

[Running] 10/10회차 측정 진행 중...
⏱️ 10회차 소요 시간: 2.1475초
🔙 하드웨어 뒤로가기 키 입력

💾 결과 저장 중...
✅ 저장 완료: work24_search_perf_10runs_20251204_145649.csv
    회차      측정시각   검색어 검색반응속도(초)   평균(초)   최소(초)   최대(초)  표준편차(초)
0    1  14:56:10  실업급여    1.5137     NaN     NaN     NaN      NaN
1    2  14:56:14  실업급여    2.1937     NaN     NaN     Na